# Исследование надёжности заёмщиков

**Задача проекта**

На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.

**Ход исследования**
 1. Обзор данных.
 2. Предобработка данных.
 3. Ответы на вопросы:
   - Есть ли зависимость между наличием детей и возвратом кредита в срок?
   - Есть ли зависимость между семейным положением и возвратом кредита в срок?
   - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
   - Как разные цели кредита влияют на его возврат в срок?

## Обзор данных

Импортируем библиотеку Pandas. 
Импортируем и прочитаем файл с данными, сохраним его в переменной `df`:


In [50]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:,.2f}'.format)


In [ ]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [27]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице:

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. 

Согласно документации к данным:
* `children` — количество детей в семье,
* `days_employed` — общий трудовой стаж в днях,
* `dob_years` — возраст клиента в годах,
* `education` — уровень образования клиента,
* `education_id` — идентификатор уровня образования,
* `family_status` — семейное положение,
* `family_status_id` — идентификатор семейного положения,
* `gender` — пол клиента,
* `income_type` — тип занятости,
* `debt` — имел ли задолженность по возврату кредитов,
* `total_income` — ежемесячный доход,
* `purpose` — цель получения кредита.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Вывод**

В каждой строке таблицы видим данные, описывающие клиента банка, данные о наличии задолженности хранятся в колонке `debt`. Для ответа на вопросы исследования нам понадобятся не все колонки, однако избавляться от них пока рано. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, некорректные типы данных, отрицательные зачения  и смешанный регистр. Также цели получения кредита прописаны не единообразно.

Для дальнейшего анализа нужно устранить проблемы в данных.

##  Предобработка данных

### Обработка пропусков

Посчитаем, сколько в таблице пропущенных значений.

In [29]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Всего видим по 2174 пропущенных значения в двух колонках. Это составляет около 10% от общего количества строк, удалять такой массив данных не следует. Пропуски в столбце с доходами совпадают с пропусками в стаже, можем предположить, что люди, которые не работали или не указали стаж соответсвенно не имели дохода. 

Пропуски в колонке `days_employed` не повлияют на результаты исследования, поэтому заменим их на нулевое значение. 

In [30]:
df['days_employed'] = df['days_employed'].fillna(0)
df.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски в `total_income` могут повлиять на определение зависимости надежности клиента от уровня дохода, поэтому заполним их медианными значениями групп, выбранных по типу занятости. 


Посчитаем медианный доход для каждой группы `income_type`. 

In [31]:
display(df.groupby("income_type")["total_income"].median())

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Заполним пропуски в `total_income` соответствующими значениями.

In [32]:
df['total_income'] = df['total_income'].fillna(df.groupby("income_type")["total_income"].transform("median"))

In [33]:
print('Количество пропусков в total_income:', df['total_income'].isna().sum())

Количество пропусков в total_income: 0


**Вывод**

В таблице не осталось пропусков, при этом все строки были сохранены. 


### Замена типа данных

В колонке `days_employed` есть отрицательные значения. Знаки минуса могли появиться в результате человеческой ошибки, например, при вводе данных ставился знак "тире". Заменим отрицательные значения, используя модуль. Выведем первые 5 значений для проверки.

In [34]:
df['days_employed'] = abs(df['days_employed'])
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Для удобства чтения таблицы преобразуем данные в колонках `days_employed` и `total_income` из `float64` в `int64`. После проверим типы данных в колонках.

In [35]:
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

Теперь таблица содержит числовые значения, удобные для чтения, сравнения и анализа. 

### Обработка дубликатов

Перед подсчетом дубликатов приведем символы в столбце `education` к нижнему регистру. 

In [36]:
df['education'] = df['education'].str.lower()

Посчитаем явные дубликаты в таблице.

In [37]:
df.duplicated().sum()

71

В таблице всего 71 повтор. Удалим их обычным методом `drop_duplicates()` и восстановим порядок индексов после удаления `reset_index()`. Затем проверим, что дубликаты отсутствуют.

In [38]:
df = df.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов после удаления:', df.duplicated().sum())

Количество дубликатов после удаления: 0


**Вывод**

Таблица готова для дальнейшего анализа. На данном этапе избавились только от явных дубликатов. Поиск неявных совпадений на данном этапе не кажется целесообразным.  Причина появления дубликатов в рамках исследования не известна, можно предположить, что они возникли при повторном вводе одинаковой информации, либо при объединении двух таблиц.

### Категоризация данных

**Определение целей кредита**


Для ответа на вопрос о целях кредита, подготовим данные, а именно, выделим цели получения кредита. Посмотрим какие категории можно выделить, выведем все значения столбца `purpose`, отсортируем по популярности.

In [39]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Можно выделить 4 категории:  *'операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы', 'получение образования'*.

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут наши категории.

In [40]:
def get_purpose(value):
  if 'автомобил' in value:
    return 'операции с автомобилем'
  if 'свадьб' in value:
    return 'проведение свадьбы'
  if 'недвиж' in value or 'жиль' in value:
    return 'операции с недвижимостью'
  if 'образован' in value:
    return 'получение образования'
  return 'другое'
    
df['purpose_category'] = df['purpose'].apply(get_purpose)
df.head(5) 


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы


Проверим, что все данные обработались верно и нет значений в категории "другое".

In [41]:
df['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

**Категоризация по количеству детей**

Посмотрим на значения колонки `children`. 

In [42]:
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Можно поделить всех клиентов на 4 категории по наличию детей: *'без детей', '1 ребенок', '2 ребенка', 'многодетные'*. 

В данных видим два выделяющихся значения 20 и -1. 
Можно предположить, что они возникли в результате опечаток. Данные значения можно исключить из анализа как выбросы, но в рамках данного проекта изменим их на 2 и 1 соответственно.

In [43]:
df[['children']] = df[['children']].replace([20, -1], [2, 1])
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Создадим новую колонку `child_category` со значениями категорий. Для этого напишем функцию, которая возвращает нужное значение в соответствии с условиями. Применим нашу функцию к столбцу `children`. 

In [44]:
def find_child(value):
    if value == 0:
        return 'без детей'
    if value == 1:
        return '1 ребенок'
    if value == 2:
        return '2 ребенка'
    
    return 'многодетные'

df['child_category'] = df['children'].apply(find_child)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,child_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,1 ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,1 ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,без детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,без детей


**Категоризация по уровню дохода**

Теперь разделим клиентов по уровню дохода. Выберем условно 3 категории: *'высокий', 'средний' и 'низкий доход'*. 

Чтобы узнать разброс значений, используем метод describe() для колонки `total_income`.

In [51]:
df['total_income'].describe()

count      21,454.00
mean      165,319.57
std        98,187.30
min        20,667.00
25%       107,623.00
50%       142,594.00
75%       195,820.25
max     2,265,604.00
Name: total_income, dtype: float64

Наишем функцию для распределения категорий дохода в зависимости от заработка. Значения меньше 25% будут означать низкий доход, значения от 25% до 75% - средний, всё что выше 75% будем считать высоким доходом.

Применим нашу функцию к столбцу `total_income` и сохраним результат в новый столбец `income_category`.

In [55]:
def income_level(value):
  low = np.percentile(df["total_income"], 25)
  middle = np.percentile(df["total_income"], 75)
  if value <= low:
        return 'низкий доход'
  if value <= middle:
        return 'средний доход'
    
  return 'высокий доход'

df['income_category'] = df['total_income'].apply(income_level)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,child_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,1 ребенок,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,1 ребенок,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,без детей,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования,многодетные,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,без детей,средний доход


In [56]:
df['income_category'].value_counts()

средний доход    10726
высокий доход     5364
низкий доход      5364
Name: income_category, dtype: int64

Создадим два отдельных «словаря», где названию категорий семейного статуса и уровня образования будет соответствовать свой id. 

Сначала выделим в отдельную таблицу колонки `education` и `education_id`

In [57]:
education_dict = df[['education', 'education_id']]
education_dict.head(5)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1


Удалим дубликаты, чтобы остались только уникальные значения.

In [58]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Заметно, что id  проставлены не логично. Высшему образованию соответствует 0, начальному 3, ученой степени 4 и т.д. В реальной жизни можно обратить на это внимание отвественных лиц.


Далее выделим в отдельную таблицу колонки `family_status` и `family_status_id`

In [59]:
family_dict = df[['family_status', 'family_status_id']]
family_dict.head(5)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


Удалим дубликаты, чтобы остались только уникальные значения.

In [60]:
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Получили готовый словарь, здесь никаких особенностей нет.

**Вывод**

В результате категоризации мы получили 3 дополнительные колонки с категориями, которые помогут построить сводные таблицы для ответов на вопросы исследования. 

Также мы выделили словари для уровня образования и семейного положения, и заметили особенность присвоения id в первом словаре.

##  Ответы на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сформируем сводную таблицу по клиентам. Значения в колонке "1" покажет какое количество клиентов с детьми и без детей имели задолженности по кредиту.

In [77]:
df_children_pivot = df.pivot_table(index='child_category', columns='debt', values='gender',  aggfunc='count')
df_children_pivot

debt,0,1
child_category,,
1 ребенок,4410,445
2 ребенка,1926,202
без детей,13028,1063
многодетные,349,31


Добавим столбец для подсчета суммы значений в строках и столбец с процентным количеством должников.

In [78]:
df_children_pivot['sum'] = df_children_pivot[0] + df_children_pivot[1]
df_children_pivot['percentage'] = df_children_pivot[1] / df_children_pivot['sum'] * 100
df_children_pivot.columns = ['No_debt', 'Debt', 'Total', '%_of_debtors']
df_children_pivot.sort_values(by='%_of_debtors', ascending=False)

,No_debt,Debt,Total,%_of_debtors
child_category,,,,
2 ребенка,1926,202,2128,9.49
1 ребенок,4410,445,4855,9.17
многодетные,349,31,380,8.16
без детей,13028,1063,14091,7.54


**Вывод**

На основе данных можно сказать, что существует зависимость между наличием детей и возвратом кредита в срок. Клиенты с 1 и 2 детьми чаще оказывались должниками. Клиенты без детей оказываются самыми надежными. Разница в показателях между этими группами составляет около 2%. Многодетные семьи также более надежны, чем люди с 1 и 2 детьми. Это можно объяснить, например, тем, что многодетные могут получать льготы или какие-то выплаты и у них освобождаются средства на погашение кредита.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сформируем сводную таблицу по клиентам. И добавим столбцы для подсчета суммы и процентов как в предыдущем пункте. 


In [79]:
df_family_pivot = df.pivot_table(index='family_status', columns='debt', values='gender',  aggfunc='count')

df_family_pivot['sum'] = df_family_pivot[0] + df_family_pivot[1]
df_family_pivot['percentage'] = df_family_pivot[1] / df_family_pivot['sum'] * 100
df_family_pivot.columns = ['No_debt', 'Debt', 'Total', '%_of_debtors']
df_family_pivot.sort_values(by='%_of_debtors', ascending=False)

,No_debt,Debt,Total,%_of_debtors
family_status,,,,
Не женат / не замужем,2536,274,2810,9.75
гражданский брак,3763,388,4151,9.35
женат / замужем,11408,931,12339,7.55
в разводе,1110,85,1195,7.11
вдовец / вдова,896,63,959,6.57


**Вывод**

Прослеживается зависимость возврата кредита в срок и семейным положением. Можно сказать, что клиенты, не состоящие в браке, чаще других оказывались должниками. Самыми надежными клиентами в выборке оказались вдовцы/вдовы, однако эта категория также самая малочисленная.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сформируем сводную таблицу по клиентам. И добавим столбцы для подсчета суммы и процентов как в предыдущем пункте.


In [80]:
df_income_pivot = df.pivot_table(index='income_category', columns='debt', values='gender',  aggfunc='count')

df_income_pivot['sum'] = df_income_pivot[0] + df_income_pivot[1]
df_income_pivot['percentage'] = df_income_pivot[1] / df_income_pivot['sum'] * 100
df_income_pivot.columns = ['No_debt', 'Debt', 'Total', '%_of_debtors']
df_income_pivot.sort_values(by='%_of_debtors', ascending=False)

,No_debt,Debt,Total,%_of_debtors
income_category,,,,
средний доход,9795,931,10726,8.68
низкий доход,4937,427,5364,7.96
высокий доход,4981,383,5364,7.14


**Вывод**

Можно сказать, что зависимость возврата кредита в срок и уровнем дохода существует. Клиенты с высоким доходом более надежны, относительно других. Процент должников в этой группе меньше всего. Разница между категориями людей с низким и средним доходами составляет около 0.7%.

### Как разные цели кредита влияют на его возврат в срок?

Сформируем сводную таблицу по клиентам. И добавим столбцы для подсчета суммы и процентов как в предыдущем пункте.


In [81]:
df_purpose_pivot = df.pivot_table(index='purpose_category', columns='debt', values='gender',  aggfunc='count')

df_purpose_pivot['sum'] = df_purpose_pivot[0] + df_purpose_pivot[1]
df_purpose_pivot['percentage'] = df_purpose_pivot[1] / df_purpose_pivot['sum'] * 100
df_purpose_pivot.columns = ['No_debt', 'Debt', 'Total', '%_of_debtors']
df_purpose_pivot.sort_values(by='%_of_debtors', ascending=False)

,No_debt,Debt,Total,%_of_debtors
purpose_category,,,,
операции с автомобилем,3903,403,4306,9.36
получение образования,3643,370,4013,9.22
проведение свадьбы,2138,186,2324,8.00
операции с недвижимостью,10029,782,10811,7.23


**Вывод**

Исходя из данных, процент возврата действительно различается в зависимости от целей кредита. Чаще других должниками оказывались клиенты, взявшие кредит на автомобиль и образование. Самыми надежными клиентами оказались покупатели недвижимости. 

## Шаг 4. Общий вывод

Мы провели исследование и установили зависимости между семейным положением клиентов, наличием детей, уровнем дохода и целями кредита на факт погашения кредита в срок.

Наибольшее влияние на погашение кредита оказывают семейное положение клиента и наличие детей. В этих категориях наблюдается заметная разница между показателями групп.

Самый высокий процент должников в группе "Не женат / не замужем"(9.7%).

Далее идет группа клиентов с 2 детьми - из них 9.5%  не смогли погасить кредит вовремя.

Следом идет группа клиентов, взявших кредит на автомобиль - 9.4% должников.

Можно сказать, что уровень дохода оказывает наименьшее влияние на возврат кредита в срок. Разница между показателями групп не значительная.
